<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/main/Datensatz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times
import json
from datetime import datetime
import numpy as np

Hier werden die Daten aus der ESPN-Api extrahiert

In [ ]:
Data_basic = pd.DataFrame()
Data_basic = pd.DataFrame(columns=[
    'date','gameID','season','types',
    'week','HomeTeam_ID','AwayTeam_ID',
    'Winner'
    ])
week = list(range(1, 19))
seasons=list(range(2015,2023))

In [ ]:
def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    rows=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      rows.append(datetime.strptime(df2['date'][0], "%Y-%m-%dT%H:%MZ").date())
      rows.append(df2['id'][0])
      rows.append(season)
      rows.append(types)
      rows.append(weeks)
      rows.append(df2['competitors'][0][0]['id'])
      rows.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          rows.append('Home')
        else:
          rows.append('Away')
      except KeyError:
        rows.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = rows

In [ ]:
for x in seasons:
  for i in week:
    infos(x,2,i)

In [ ]:
play_data = pd.DataFrame()
play_data = pd.DataFrame(columns=['gameID', 'playID', 'awayScore', 'homeScore', 'scoreValue',
                         'statYardage', 'quarter', 'clock', 'typeid','type_text', 'team_play'
                         , 'start_down', 'start_distance', 'start_yardlinie'
                         ,'start_yardsToEndzone', 'end_down', 'end_distance', 'end_yardlinie'
                         ,'end_yardsToEndzone'])


In [ ]:
def playid(GameID):
    url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
    response_drive = r.get(url)
    drive_data = response_drive.json()
    drive_df = json_normalize(drive_data['items'])
    for index, row in drive_df.iterrows():
      for detail in range(len(row['plays.items'])):
        gameID = GameID
        playID = row['plays.items'][detail]['id']
        awayScore = row['plays.items'][detail]['awayScore']
        homeScore = row['plays.items'][detail]['homeScore']
        scoreValue = row['plays.items'][detail]['scoreValue']
        statYardage = row['plays.items'][detail]['statYardage']
        clock = datetime.timedelta(seconds=row['plays.items'][detail]['clock']['value'])
        quarter = row['plays.items'][detail]['period']['number']
        type_id = row['plays.items'][detail]['type']['id']
        type_text = row['plays.items'][detail]['type']['text']
        try:
          team_play = re.search(r"teams/(\d+)", row['plays.items'][detail]['start']['team']['$ref']).group(1)
        except KeyError:
          team_play = 'NoTeam'
        start_down = row['plays.items'][detail]['start']['down']
        start_distance = row['plays.items'][detail]['start']['distance']
        start_yardlinie = row['plays.items'][detail]['start']['yardLine']
        start_yardsToEndzone = row['plays.items'][detail]['start']['yardsToEndzone']
        end_down = row['plays.items'][detail]['end']['down']
        end_distance = row['plays.items'][detail]['end']['distance']
        end_yardlinie = row['plays.items'][detail]['end']['yardLine']
        end_yardsToEndzone = row['plays.items'][detail]['end']['yardsToEndzone']
        data = [gameID,playID,awayScore,homeScore,scoreValue,statYardage, quarter, clock, type_id, type_text,
                team_play,start_down, start_distance, start_yardlinie, start_yardsToEndzone,
                end_down, end_distance,end_yardlinie,end_yardsToEndzone]
        play_data.loc[len(play_data)] = data

      #print(row)

In [ ]:
for game in Data_basic['GameID']:
  playid(game)

Hier werden die aktuellen Elo-Zahlen extrahiert

In [ ]:
teams_name_elo = pd.DataFrame(columns=['team_id', 'team_fullname', 'team_name'])
year = '2015'
url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{year}/teams'
response_teams = r.get(url)
teams_data = response_teams.json()
teams_df = json_normalize(teams_data['items'])
for index, row in teams_df.iterrows():
    api = row['$ref']
    team_name = r.get(api)
    teamname_json = team_name.json()
    team = teamname_json['abbreviation'].lower()
    full_name = teamname_json['displayName']
    data = [teamname_json['id'], full_name, team]
    teams_name_elo.loc[len(teams_name_elo)] = data

In [ ]:
elonumbers = []
for teams in teams_name_elo['team_name']:
    elo_api = f'https://projects.fivethirtyeight.com/complete-history-of-the-nfl/data/{teams}.json'
    elo = r.get(elo_api)
    elo_json = elo.json()
    elonumbers.append(elo_json)
teams_name_elo['elo_score_historie'] = elonumbers
print(teams_name_elo)

In [ ]:
data = teams_name_elo
data.rename(columns={'team_id': 'HomeTeam_ID'}, inplace=True)
data.rename(columns={'team_fullname': 'Home_fullname'}, inplace=True)
data.rename(columns={'team_name': 'Home_teamname'}, inplace=True)
data.rename(columns={'elo_score_historie': 'Home_elo_score_historie'}, inplace=True)
game_data_1 = pd.merge(Data_basic, data, how='left', on='HomeTeam_ID')

In [ ]:
data.rename(columns={'HomeTeam_ID': 'AwayTeam_ID'}, inplace=True)
data.rename(columns={'Home_fullname': 'Away_fullname'}, inplace=True)
data.rename(columns={'team_name': 'Away_teamname'}, inplace=True)
data.rename(columns={'Home_elo_score_historie': 'Away_elo_score_historie'}, inplace=True)
game_data = pd.merge(game_data_1, data, how='left', on='AwayTeam_ID')

Hier wird die richtige Elozahl für die jeweilige Woche und Saison herausgesucht und abgespeichert

In [ ]:
for index, row in game_data.iterrows():
    json_string = row['Home_elo_score_historie'].replace("'", "\"")
    json_elo = json.loads(json_string)
    for element in json_elo['value']:
        try:
            week_string = element['w']
            date_string = element['d'][-4:]
        except KeyError:
            week_string = 'Week 1000'
            date_string = '2035'
        week = 'Week ' + str(row['week'])
        date = row['date'][:4]
        if date_string == date and week_string == week:
            target_element = element['y']
    previous_element = None
    for element in json_elo['value']:
        if element['y'] == target_element:
            if previous_element is not None:
                game_data.at[index, 'elo_score_home'] = previous_element['y']
            break
        previous_element = element

In [ ]:
for index, row in game_data.iterrows():
    json_string = row['Away_elo_score_historie'].replace("'", "\"")
    json_elo = json.loads(json_string)
    for element in json_elo['value']:
        try:
            week_string = element['w']
            date_string = element['d'][-4:]
        except KeyError:
            week_string = 'Week 1000'
            date_string = '2035'
        week = 'Week ' + str(row['week'])
        date = row['date'][:4]
        if date_string == date and week_string == week:
            target_element = element['y']
    previous_element = None
    for element in json_elo['value']:
        if element['y'] == target_element:
            if previous_element is not None:
                game_data.at[index, 'elo_score_away'] = previous_element['y']
            break
        previous_element = element

Hier wird die Zuscherquote extrahiert

In [ ]:
def stadium(GameID):
    url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}'
    response = r.get(url)
    json_data = response.json()
    data = json_normalize(json_data)
    try:
        attendence = data['competitions'][0][0]['attendance']
        capacity = data['competitions'][0][0]['venue']['capacity']
        statdium_attendence = attendence/capacity
        return statdium_attendence
    except KeyError:
        return 'NaN'

In [ ]:
game_data.drop('elo_score', axis=1, inplace=True)
for index, row in game_data.iterrows():
    game_data.at[index, 'attendence'] = stadium(row['GameID'])

Hier wird der finale Datensatz gebildet

In [ ]:
play_data.rename(columns={'gameID': 'GameID'},inplace=True)
full_data = pd.merge(play_data, game_data, how='left', on='GameID' )

In [ ]:
for index, row in full_data.iterrows():
  if row['team_play']==str(row['HomeTeam_ID']):
    full_data.at[index, 'team_play']=str(row['Home_teamname'])
  elif row['team_play']==str(row['AwayTeam_ID']):
    full_data.at[index, 'team_play']=str(row['Away_teamname'])


In [ ]:
full_data.drop('Away_fullname',axis=1,inplace=True)
full_data.drop('Away_fullname',axis=1,inplace=True)
full_data['clock'] = pd.to_timedelta(full_data['clock'])
full_data.drop('date',axis=1, inplace=True)

In [ ]:
play_data = full_data

Hier wird die verbleibende Zeit im Spiel berechnet

In [ ]:
play_data['clock'] = play_data['clock'].apply(lambda x: timedelta(days=int(x.split()[0]), hours=int(x.split()[2].split(':')[0]), minutes=int(x.split()[2].split(':')[1]), seconds=int(x.split()[2].split(':')[2])).total_seconds() / 60.0)
play_data['time_remaining'] = np.where(play_data['quarter'] == 1, play_data['clock'] + 45, play_data['clock'])
play_data['time_remaining'] = np.where(play_data['quarter'] == 2, play_data['clock'] + 30, play_data['time_remaining'])
play_data['time_remaining'] = np.where(play_data['quarter'] == 3, play_data['clock'] + 15, play_data['time_remaining'])
play_data['time_remaining'] = np.where(play_data['quarter'] == 4, play_data['clock'], play_data['time_remaining'])
play_data['time_remaining'] = np.where(play_data['quarter'] == 5, play_data['clock'], play_data['time_remaining'])
play_data.loc[play_data['Winner'] == 'Home', 'Winner'] = 1
play_data.loc[play_data['Winner'] == 'Away', 'Winner'] = 0
play_data['Winner'] = play_data['Winner'].astype(int)

Hier werden die restlichen Features gebildet

In [ ]:
def calculate_scoring_game(point_diff):
    if point_diff <= 7:
        return 1
    elif 7 < point_diff < 15:
        return 2
    elif 15 <= point_diff < 22:
        return 3
    else:
        return 4

In [ ]:
for index,row in play_data.iterrows():
  if row['homeScore'] > row['awayScore']:
    play_data.at[index, 'leading_Team'] = row['Home_teamname']
  elif row['homeScore'] < row['awayScore']:
    play_data.at[index, 'leading_Team'] = row['Away_teamname']
  else:
    play_data.at[index, 'leading_Team'] = 'tie'
  point_diff = abs(row['homeScore'] - row['awayScore'])
  play_data.at[index, 'point_difference'] = point_diff

play_data['scoring_game'] = play_data['point_difference'].apply(calculate_scoring_game)

Der Finale Datensatz befindet sich im Release 1!